# Exercise 6.14

In [2]:
import pathlib
import subprocess
import numpy as np
import pandas as pd
from IPython.display import display, Math
from scipy import linalg as la
from scipy import stats
import TwoWayAnova
import chapter_6_utils

In [3]:
def load_data():
    f = pathlib.WindowsPath(r'..\..\Data\Exercise6.14.xlsx')
    return pd.read_excel(f)

In [4]:
df = load_data()

## (a)

In [5]:
fit = TwoWayAnova.TwoWayManova(df, factor1_col='Factor1', factor2_col='Factor2', var_cols=['x1', 'x2'])

In [6]:
fit.x.display_obs_breakdown(['1.05cm','2.5cm','2.2cm','1.1cm','1.5cm','1.75cm'])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## (b)

In [7]:
fit.display_manova_table()

<IPython.core.display.Math object>

## (c)

In [8]:
import scipy.linalg as la
from scipy import stats

## Test for interaction

In [9]:
alpha = 0.05
Lmbda_star_int = la.det(fit.manova.W)/(la.det(fit.manova.I + fit.manova.W))
display(Math(r'\Lambda_{1}^{\star}'
             '='
             r'\frac{\left| \text{SSP}_{\text{res}} \right|}{\left| \text{SSP}_{\text{int}} + \text{SSP}_{\text{res}} \right|}'
             '='
             fr'\frac{{ {la.det(fit.manova.W):.4f} }}{{ {la.det(fit.manova.I + fit.manova.W):.4f} }}'
             '='
             f'{Lmbda_star_int:.4f}'
             ))

<IPython.core.display.Math object>

In [14]:
g = fit._design.metadata.g
b = fit._design.metadata.b
p = fit._design.metadata.p
n = fit._design.metadata.n
F_int = ((1-Lmbda_star_int)/Lmbda_star_int)*((g*b*(n-1)-p+1)/2) / ((np.abs((g-1)*(b-1) - p) + 1)/2)
display(Math('F'
             '='
             r'\left( \frac{1 - \Lambda^{\star}}{\Lambda^{\star}} \right)'
             r'\frac{(gb(n-1) - p + 1)/2}{(|(g-1)(b-1) - p| + 1)/2}'
             '='
             fr'\left( \frac{{ 1 - {Lmbda_star_int:.4f} }}{{ {Lmbda_star_int:.4f} }} \right)'
             fr'\frac{{ ({g}({b})({n-1}) - {p} + 1) /2 }}{{ (|{g-1}({b-1}) - {p}| + 1)/2 }}'
             '='
             f'{F_int:.2f}'
             ))

<IPython.core.display.Math object>

In [12]:
v1_int = np.abs((g-1)*(b-1) - p) + 1
display(Math(r'\nu_{1}'
             '='
             r'\left| (g-1)(b-1) - p \right| + 1'
             '='
             fr'\left| {g-1}({b-1}) - {p} \right| + 1'
             '='
             f'{v1_int}'
             ))

<IPython.core.display.Math object>

In [13]:
v2_int = g*b*(n-1) - p + 1
display(Math(r'\nu_{2}'
             '='
             'gb(n - 1) - p + 1'
             '='
             fr'{g}({b})({n-1}) - {p} + 1'
             '='
             f'{v2_int}'
             ))

<IPython.core.display.Math object>

In [15]:
alpha = 0.05
f_crit_int = stats.f.ppf(1-alpha,v1_int,v2_int)
display(Math(r'F_{\nu_{1}, \nu_{2}}(\alpha)'
             '='
             fr'F_{{ {v1_int}, {v2_int} }}({alpha})'
             '='
             f'{f_crit_int:.2f}'
             ))

<IPython.core.display.Math object>

In [21]:
if F_int > f_crit_int:
    display(Math(fr'\text{{We have that }} F = {F_int:.2f} > F_{{\text{{crit}}}} = F_{{ {v1_int}, {v2_int} }} \left( {alpha} \right) = '
                 fr'{f_crit_int:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\gamma}_{11} = \bm{\gamma}_{12} = \bm{\gamma}_{13} = \bm{\gamma}_{14} = '
                 r'\bm{\gamma}_{21} = \bm{\gamma}_{22} = \bm{\gamma}_{23} = \bm{\gamma}_{24} = '
                 r'\bm{\gamma}_{31} = \bm{\gamma}_{32} = \bm{\gamma}_{33} = \bm{\gamma}_{34} = \textbf{0}'
                 ))
else:
    display(Math(fr'\text{{We have that }} F = {F_int:.2f} < F_{{\text{{crit}}}} = F_{{ {v1_int}, {v2_int} }} \left( {alpha} \right) = '
                fr'{f_crit_int:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\gamma}_{11} = \bm{\gamma}_{12} = \bm{\gamma}_{13} = \bm{\gamma}_{14} = '
                r'\bm{\gamma}_{21} = \bm{\gamma}_{22} = \bm{\gamma}_{23} = \bm{\gamma}_{24} = '
                r'\bm{\gamma}_{31} = \bm{\gamma}_{32} = \bm{\gamma}_{33} = \bm{\gamma}_{34} = \textbf{0}'

                ))

<IPython.core.display.Math object>

No interaction term. We're left with the additive model.

## Test for factor 1 effects

In [24]:
Lmbda_star_fac1 = la.det(fit.manova.W) / la.det(fit.manova.B1 + fit.manova.W)
display(Math(r'\Lambda_{1}^{\star}'
             '='
             r'\frac{\left| \text{SSP}_{\text{res}} \right|}{\left| \text{SSP}_{\text{fac1}} + \text{SSP}_{\text{res}} \right|}'
             '='
             fr'\frac{{ {la.det(fit.manova.W):.4f} }}{{ {la.det(fit.manova.B1 + fit.manova.W):.4f} }}'
             '='
             f'{Lmbda_star_fac1:.4f}'
             ))

<IPython.core.display.Math object>

In [25]:
F_fac1 = ((1-Lmbda_star_fac1)/Lmbda_star_fac1)*((g*b*(n-1)-p+1)/2) / ((np.abs((g-1) - p) + 1)/2)
display(Math('F_{1}'
             '='
             r'\left( \frac{1 - \Lambda_{1}^{\star}}{\Lambda_{1}^{\star}} \right)'
             r'\frac{(gb(n-1) - p + 1)/2}{(|(g-1) - p| + 1)/2}'
             '='
             fr'\left( \frac{{ 1 - {Lmbda_star_fac1:.4f} }}{{ {Lmbda_star_fac1:.4f} }} \right)'
             fr'\frac{{ ({g}({b})({n-1}) - {p} + 1) /2 }}{{ (|{g-1} - {p}| + 1)/2 }}'
             '='
             f'{F_fac1:.2f}'
             ))

<IPython.core.display.Math object>

In [26]:
v1_fac1 = np.abs((g-1) - p) + 1
display(Math(r'\nu_{1}'
             '='
             '|(g-1) - p| + 1'
             '='
             f'|{g-1} - {p}| + 1'
             '='
             f'{v1_fac1}'
             ))

<IPython.core.display.Math object>

In [27]:
v2_fac1 = g*b*(n-1)-p+1
display(Math(r'\nu_{2}'
             '='
             'gb(n-1) - p + 1'
             '='
             f'{g*b*(n-1)} - {p} + 1'
             '='
             f'{v2_fac1}'
             ))

<IPython.core.display.Math object>

In [28]:
f_crit_fac1 = stats.f.ppf(1-alpha,v1_fac1,v2_fac1)
display(Math(r'F_{\nu_{1}, \nu_{2}}(\alpha)'
             '='
             fr'F_{{ {v1_fac1}, {v2_fac1} }}({alpha})'
             '='
             f'{f_crit_fac1:.2f}'
             ))

<IPython.core.display.Math object>

In [29]:
if F_fac1 > f_crit_fac1:
    display(Math(fr'\text{{We have that }} F_{{1}} = {F_fac1:.2f} > F_{{\text{{crit}}}} = F_{{ {v1_fac1}, {v2_fac1} }} \left( {alpha} \right) = '
                 fr'{f_crit_fac1:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\tau}_{1} =\bm{\tau}_{2} = \bm{\tau}_{3} = \textbf{0}'))
else:
    display(Math(fr'\text{{We have that }} F_{{1}} = {F_fac1:.2f} < F_{{\text{{crit}}}} = F_{{ {v1_fac1}, {v2_fac1} }} \left( {alpha} \right) = '
                fr'{f_crit_fac1:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\tau}_{1} =\bm{\tau}_{2} = \bm{\tau}_{3} = \textbf{0}'))

<IPython.core.display.Math object>

## Test for factor 2 effects

In [32]:
Lmbda_star_fac2 = la.det(fit.manova.W) / la.det(fit.manova.B2 + fit.manova.W)
display(Math(r'\Lambda_{2}^{\star}'
             '='
             r'\frac{\left| \text{SSP}_{\text{res}} \right|}{\left| \text{SSP}_{\text{fac2}} + \text{SSP}_{\text{res}} \right|}'
             '='
             fr'\frac{{ {la.det(fit.manova.W):.4f} }}{{ {la.det(fit.manova.B2 + fit.manova.W):.4f} }}'
             '='
             f'{Lmbda_star_fac2:.4f}'
             ))

<IPython.core.display.Math object>

In [33]:
F_fac2 = ((1-Lmbda_star_fac2)/Lmbda_star_fac2)*((g*b*(n-1)-p+1)/2) / ((np.abs((b-1) - p) + 1)/2)
display(Math('F_{2}'
             '='
             r'\left( \frac{1 - \Lambda_{2}^{\star}}{\Lambda_{2}^{\star}} \right)'
             r'\frac{(gb(n-1) - p + 1)/2}{(|(b-1) - p| + 1)/2}'
             '='
             fr'\left( \frac{{ 1 - {Lmbda_star_fac2:.4f} }}{{ {Lmbda_star_fac2:.4f} }} \right)'
             fr'\frac{{ ({g}({b})({n-1}) - {p} + 1) /2 }}{{ (|{b-1} - {p}| + 1)/2 }}'
             '='
             f'{F_fac2:.2f}'
             ))

<IPython.core.display.Math object>

In [34]:
v1_fac2 = np.abs((b-1) - p) + 1
display(Math(r'\nu_{1}'
             '='
             '|(b-1) - p| + 1'
             '='
             f'|{b-1} - {p}| + 1'
             '='
             f'{v1_fac2}'
             ))

<IPython.core.display.Math object>

In [35]:
v2_fac2 = g*b*(n-1)-p+1
display(Math(r'\nu_{2}'
             '='
             'gb(n-1) - p + 1'
             '='
             f'{g*b*(n-1)} - {p} + 1'
             '='
             f'{v2_fac2}'
             ))

<IPython.core.display.Math object>

In [36]:
f_crit_fac2 = stats.f.ppf(1-alpha,v1_fac2,v2_fac2)
display(Math(r'F_{\nu_{1}, \nu_{2}}(\alpha)'
             '='
             fr'F_{{ {v1_fac2}, {v2_fac2} }}({alpha})'
             '='
             f'{f_crit_fac2:.2f}'
             ))

<IPython.core.display.Math object>

In [347]:
if F_fac2 > f_crit_fac2:
    display(Math(fr'\text{{We have that }} F_{{2}} = {F_fac2:.2f} > F_{{\text{{crit}}}} = F_{{ {v1_fac2}, {v2_fac2} }} \left( {alpha} \right) = '
                 fr'{f_crit_fac2:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\beta}_{1} = \bm{\beta}_{2} = \bm{\beta}_{3} = \bm{\beta}_{4} = \textbf{0}'))
else:
    display(Math(fr'\text{{We have that }} F_{{2}} = {F_fac2:.2f} < F_{{\text{{crit}}}} = F_{{ {v1_fac2}, {v2_fac2} }} \left( {alpha} \right) = '
                fr'{f_crit_fac2:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\beta}_{1} = \bm{\beta}_{2} = \bm{\beta}_{3} = \bm{\beta}_{4} = \textbf{0}'))

<IPython.core.display.Math object>

## (d)

In [215]:
from collections import namedtuple

In [412]:
def bonferroni_two_way(a_fit: TwoWayAnova, a_factor: str, alpha: float) -> dict[str, list[np.ndarray]]:
    # The formulas used here is from (6-70) on pages 317 and 318.
    # Slice a row out of the tau tensor. Make sure they're columns. There's a column for every variable. The variable is the first arg in shape.
    g = a_fit._design.metadata.g
    b = a_fit._design.metadata.b
    p = a_fit._design.metadata.p
    n = a_fit._design.metadata.n

    def compute_bonf_ci(trt_levels1: int, trt_levels2: int, factor: np.ndarray) -> list[np.ndarray]:
        ci_dict = {}
        v = trt_levels1 * trt_levels2 * (n - 1)
        t_val = stats.t.ppf(1-(alpha/(2*p*trt_levels1*(trt_levels1-1))), df=v)
        pairs, C = create_contrast_matrix(trt_levels1)
        ci_dict.update({'pairs': pairs})
        # C = (np.diag(np.ones((1,trt_levels1))[0], 0) + np.diag(-np.ones((1,trt_levels1-1))[0], 1))[:-1,:]
        tau_diff_matrix = C @ factor.T
        E = np.diag(a_fit.manova.W)*np.ones((trt_levels1-1,p))
        
        # Loop through each of the p variables and compute the CI.
        for i in range(tau_diff_matrix.shape[1]):
            # tau_diff = tau_diff_matrix[:, i].reshape(trt_levels1-1,1)
            height = tau_diff_matrix[:, i].shape[0]
            tau_diff = tau_diff_matrix[:, i].reshape(height,1)
            # E_ii = E[:, i].reshape(trt_levels1-1,1)
            E_ii = E[0,i] * np.ones((height,1))
            ci = tau_diff + np.array([-1, 1]) * t_val * np.sqrt(E_ii * 1/v * 2/(trt_levels2*n))
            ci_dict.update({f'x{i+1}': ci})
        return ci_dict
    
    BonferroniCI = namedtuple('BonferroniCI', ['Factor1', 'Factor2'])
    return BonferroniCI(
        # All columns are the same for factor1, so pick first column.
        Factor1=compute_bonf_ci(g, b, a_fit.x.anova_values.Trt1Effect[:,:,0]),
        # All rows are the same for factor2, so pick first row.
        Factor2=compute_bonf_ci(b, g, a_fit.x.anova_values.Trt2Effect[:,0,:])
        )

In [443]:
all_bonf_pairwise = bonferroni_two_way(fit, True, 0.05)

In [445]:
# all_bonf_pairwise 

In [348]:
import itertools

In [393]:
def create_contrast_matrix(n):
    pair_indices = list(itertools.combinations(range(n), 2))

    # Build the contrast matrix:
    # For each pair (i, j), create a contrast vector with 1 at i, -1 at j, and 0 elsewhere.
    contrast_list = []
    for i, j in pair_indices:
        contrast = np.zeros(n)
        contrast[i] = 1
        contrast[j] = -1
        contrast_list.append(contrast)
    return pair_indices, np.vstack(contrast_list)

In [446]:
def display_two_way_manova_bonf(bonf_output: namedtuple) -> None:
    for factor_name, variables in bonf_output._asdict().items():
        variables = variables.copy()
        print(factor_name)
        pairs = variables.pop('pairs')
        # print(pairs)
        for i, (variable, cis) in enumerate(variables.items()):
            # print(variable)
            for j in range(cis.shape[0]):
                ci = cis[j,:]
                ell = pairs[j][0]+1
                m = pairs[j][1]+1
                display(Math(fr'x_{i+1}:'))
                display(Math(fr'\qquad\qquad\ell = {ell}, m = {m}, i = {i+1}'))
                display(Math(fr'\qquad\qquad\tau_{{ {ell}, {i+1} }} - \tau_{{ {m}, {i+1} }}'
                             r'\pm '
                             r't_{v}'
                             r'\left( \frac{\alpha}{pg(g-1)} \right)'
                             fr'\sqrt{{ \frac{{ E_{{ {i+1}, {i+1} }} }}{{ v }} \frac{{ 2 }}{{ bn }} }}'
                            '='
                            f'[{ci[0]:.4f}, {ci[1]:.4f}]'
                            ))

In [447]:
display_two_way_manova_bonf(all_bonf_pairwise)

Factor1


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Notice how almost every confidence interval contains 0. The only exception is the pairwise comparison of $\tau_{1,1} - \tau_{3,1}$, so there is a difference in means between level 1 and level 3 for Factor 1 variable 1. This is what's causing Factor 1 main effect to be significant. All difference confidence intervals contain 0 for Factor 2, which is consistent with there being no significant effects for Factor 2.

## MANOVA results from R

Using R we can get the same output as SAS. MANOVA in Python unforunately doesn't match up with everyone else.

In [345]:
result = subprocess.run([r'C:\Program Files\R\R-4.4.2\bin\Rscript', '--vanilla', r'..\..\r\chapter-6\Question-6-14.R'], capture_output=True, text=True)
print(result.stdout)


Type II MANOVA Tests:

Sum of squares and products for error:
    x1  x2
x1 312 -84
x2 -84 400

------------------------------------------
 
Term: Factor1 

Sum of squares and products for the hypothesis:
    x1  x2
x1 496 184
x2 184 208

Multivariate Tests: Factor1
                 Df test stat  approx F num Df den Df     Pr(>F)    
Pillai            2 0.8993650  4.902797      4     24 0.00493493 ** 
Wilks             2 0.2446557  5.619493      4     22 0.00284380 ** 
Hotelling-Lawley  2 2.4987091  6.246773      4     20 0.00197672 ** 
Roy               2 2.2353666 13.412200      2     12 0.00087188 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

------------------------------------------
 
Term: Factor2 

Sum of squares and products for the hypothesis:
   x1 x2
x1 36 24
x2 24 36

Multivariate Tests: Factor2
                 Df test stat  approx F num Df den Df  Pr(>F)
Pillai            3 0.2099806 0.4692252      6     24 0.82422
Wilks             3 0.7948801 